In [1]:
#Importing libraries
import zipfile 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
import os
import matplotlib.pyplot as plt#visualization
from PIL import  Image
%matplotlib inline
import pandas as pd
import seaborn as sns#visualization
import itertools
import warnings
warnings.filterwarnings("ignore")
import io
import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization
import plotly.express as px

start_time = pd.datetime.now()

In [44]:
data = pd.read_csv('data.csv')

In [45]:
data=data.drop(['Unnamed: 0'], axis=1)

In [46]:
data.head()

,channel,cnt,date_time,hotel_cluster,hotel_continent,hotel_country,hotel_market,is_booking,is_mobile,is_package,...,srch_children_cnt,srch_ci,srch_co,srch_destination_id,srch_destination_type_id,srch_rm_cnt,user_id,user_location_city,user_location_country,user_location_region
0,1,1,2014-06-12 22:56:41,51,2,50,628,0,0,1,...,0,2014-07-09 00:00:00,2014-07-13 00:00:00,8250,1,1,461215,20706,66,153
1,9,1,2014-07-01 02:29:47,38,4,196,2027,0,0,1,...,0,2014-09-14 00:00:00,2014-09-17 00:00:00,8275,1,1,850200,45619,68,330
2,9,2,2013-05-26 20:32:08,6,2,198,379,0,0,0,...,0,2013-06-02 00:00:00,2013-06-04 00:00:00,25928,1,1,1197652,41495,205,354
3,9,2,2014-10-24 13:30:01,28,2,50,674,0,0,0,...,0,2014-11-08 00:00:00,2014-11-10 00:00:00,12483,5,1,734197,2428,66,196
4,9,1,2013-05-27 00:11:40,38,5,73,1722,0,0,1,...,0,2013-06-07 00:00:00,2013-06-16 00:00:00,15582,1,1,710820,28661,24,67


In [49]:
#change datatime data type
datetime_var = ['date_time','srch_ci','srch_co']
def change_datatype():
    for i in datetime_var:
        data[i] = data[i].astype('datetime64[ns]')
    return data
 
data = change_datatype()

In [50]:
data.dtypes

channel                               int64
cnt                                   int64
date_time                    datetime64[ns]
hotel_cluster                         int64
hotel_continent                       int64
hotel_country                         int64
hotel_market                          int64
is_booking                            int64
is_mobile                             int64
is_package                            int64
orig_destination_distance           float64
posa_continent                        int64
rec_hotel_cluster                     int64
site_name                             int64
srch_adults_cnt                       int64
srch_children_cnt                     int64
srch_ci                      datetime64[ns]
srch_co                      datetime64[ns]
srch_destination_id                   int64
srch_destination_type_id              int64
srch_rm_cnt                           int64
user_id                               int64
user_location_city              

In [ ]:
## Feature Engineering

In [51]:
data['checkin_duration']=(data['srch_co']-data['srch_ci']).dt.days

In [52]:
data['days_since_sr_login']=(data['srch_co']-data['date_time']).dt.days

In [53]:
data['weekend']=data['date_time'].apply(lambda x: 1 if x.dayofweek==5 or x.dayofweek==6 else 0)

In [54]:
from datetime import datetime
data['date_time_year'] = pd.DatetimeIndex(data['date_time']).year
data['date_time_month'] = pd.DatetimeIndex(data['date_time']).month


In [55]:
data['srch_ci_year'] = pd.DatetimeIndex(data['srch_ci']).year
data['srch_ci_month'] = pd.DatetimeIndex(data['srch_ci']).month

data['srch_co_year'] = pd.DatetimeIndex(data['srch_co']).year
data['srch_co_month'] = pd.DatetimeIndex(data['srch_co']).month

In [56]:
data = data.drop(['date_time','srch_ci','srch_co'],axis=1)

In [58]:
group=['srch_destination_id','hotel_country','hotel_market','rec_hotel_cluster']
for i in group:
    data[str(i)+'_booking_mean'] = data.groupby([i])['is_booking'].transform(
    lambda x: x.mean())
    data[str(i)+'_booking_sum'] = data.groupby([i])['is_booking'].transform(
    lambda x: x.sum())


In [60]:
data['group_booking_mean'] = data.groupby(['srch_destination_id','hotel_country','rec_hotel_cluster'
                                           ,'hotel_market'])['is_booking'].transform(
    lambda x: x.mean())

data['group_booking_sum'] = data.groupby(['srch_destination_id','hotel_country','rec_hotel_cluster',
                                          'hotel_market'])['is_booking'].transform(
    lambda x: x.sum())

In [61]:
data.head()

,channel,cnt,hotel_cluster,hotel_continent,hotel_country,hotel_market,is_booking,is_mobile,is_package,orig_destination_distance,...,srch_destination_id_booking_mean,srch_destination_id_booking_sum,hotel_country_booking_mean,hotel_country_booking_sum,hotel_market_booking_mean,hotel_market_booking_sum,rec_hotel_cluster_booking_mean,rec_hotel_cluster_booking_sum,group_booking_mean,group_booking_sum
0,1,1,51,2,50,628,0,0,1,254.716500,...,0.072624,972,0.089363,16091,0.073171,1308,0.104506,610,0.050000,7
1,9,1,38,4,196,2027,0,0,1,1877.175685,...,0.057540,29,0.085953,134,0.069876,45,0.065149,166,0.040000,1
2,9,2,6,2,198,379,0,0,0,143.387800,...,0.107438,13,0.109852,1948,0.111111,14,0.102819,496,0.000000,0
3,9,2,28,2,50,674,0,0,0,195.738700,...,0.170213,32,0.089363,16091,0.123905,99,0.104939,529,0.176471,3
4,9,1,38,5,73,1722,0,0,1,4927.422533,...,0.086957,4,0.088000,11,0.072727,4,0.052127,185,0.000000,0


In [ ]:
## Modeling

In [62]:
df = data[data['is_booking']==1]
len(df)

29749

In [63]:
X = df.drop(['user_id', 'hotel_cluster', 'is_booking'], axis=1)
y = df.hotel_cluster

In [ ]:
##data scaling

In [27]:
from sklearn import preprocessing

In [64]:
X_scaled = preprocessing.scale(X)

In [ ]:
##PCA

In [65]:
X_PCA = X_scaled.copy()


In [66]:
from sklearn.decomposition import PCA
pca = PCA(n_components=25)
pca.fit(X_PCA)
print(pca.explained_variance_ratio_)
X_PCA = pca.transform(X_PCA)

[0.09325254 0.0899482  0.07653516 0.0631239  0.04334784 0.04244158
 0.03994234 0.03760989 0.03432674 0.03363124 0.03084815 0.02902339
 0.02805223 0.02620672 0.02549476 0.02504352 0.02474634 0.02300503
 0.02213654 0.02171444 0.0210938  0.01903377 0.01741798 0.01680963
 0.01542865]


In [116]:
pca_varaince = list(pca.explained_variance_ratio_)
np.sum(pca_varaince )

0.9002143575467236

In [67]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn import svm

In [33]:
##Naive Bayes

In [68]:
from sklearn.naive_bayes import GaussianNB
clf = make_pipeline(preprocessing.StandardScaler(), GaussianNB(priors=None))
np.mean(cross_val_score(clf, X_PCA, y, cv=10))

0.13093833916107644

In [69]:
clf = make_pipeline(preprocessing.StandardScaler(), GaussianNB(priors=None))
np.mean(cross_val_score(clf, X, y, cv=10))

0.0476786047388489

In [70]:
clf = make_pipeline(preprocessing.StandardScaler(), GaussianNB(priors=None))
np.mean(cross_val_score(clf, X_scaled, y, cv=10))

0.0476786047388489

In [ ]:
##K-Nearest Neighbors Classifier

In [71]:
from sklearn.neighbors import KNeighborsClassifier
clf = make_pipeline(preprocessing.StandardScaler(), KNeighborsClassifier(n_neighbors=5))
np.mean(cross_val_score(clf, X_scaled, y, cv=10, scoring='accuracy'))

0.09326500519977152

In [72]:
clf = make_pipeline(preprocessing.StandardScaler(), KNeighborsClassifier(n_neighbors=5))
np.mean(cross_val_score(clf, X, y, cv=10, scoring='accuracy'))

0.09326500519977152

In [73]:
clf = make_pipeline(preprocessing.StandardScaler(), KNeighborsClassifier(n_neighbors=5))
np.mean(cross_val_score(clf, X_PCA, y, cv=10, scoring='accuracy'))

0.08142691088532247

In [ ]:
##Random Forest Classifier

In [74]:
clf = make_pipeline(preprocessing.StandardScaler(), 
                    RandomForestClassifier(n_estimators=273,max_depth=10,random_state=0))
np.mean(cross_val_score(clf, X_scaled, y, cv=10, scoring='accuracy'))


0.5055700673359136

In [75]:
clf = make_pipeline(preprocessing.StandardScaler(), 
                    RandomForestClassifier(n_estimators=273,max_depth=10,random_state=0))
np.mean(cross_val_score(clf, X, y, cv=10, scoring='accuracy'))

0.5055700673359136

In [98]:
X_fi = X.dropna()

In [103]:
#feature importance
from sklearn.model_selection import train_test_split
X2 = df.drop(['user_id', 'is_booking'], axis=1)
target = ['hotel_cluster']
train,test = train_test_split(X2,test_size =.25 ,random_state = 111)
cols    = [i for i in X2.columns if i not in target]
train_X = train[cols]
train_Y = train[target]
test_X  = test[cols]
test_Y  = test[target]

In [106]:
rf = RandomForestClassifier(n_estimators=273,max_depth=10,random_state=0) 
rf.fit(train_X, train_Y) 
rf.score(test_X, test_Y)
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = train_X.columns,
                                    columns=['importance']).sort_values('importance',
                                                                        ascending=False)

In [107]:
feature_importances

,importance
rec_hotel_cluster_booking_mean,0.211335
rec_hotel_cluster,0.185523
rec_hotel_cluster_booking_sum,0.177068
group_booking_sum,0.054899
hotel_market_booking_sum,0.043229
group_booking_mean,0.029440
hotel_market_booking_mean,0.026996
srch_destination_id_booking_sum,0.024119
hotel_country_booking_sum,0.023473
srch_destination_id_booking_mean,0.023430


In [110]:
test_X['predicted_hotel_cluster']=rf.predict(test_X)
test_X['actaul_hotel_cluster'] = test_Y

In [111]:
test_X

,channel,cnt,hotel_continent,hotel_country,hotel_market,is_mobile,is_package,orig_destination_distance,posa_continent,rec_hotel_cluster,...,hotel_country_booking_mean,hotel_country_booking_sum,hotel_market_booking_mean,hotel_market_booking_sum,rec_hotel_cluster_booking_mean,rec_hotel_cluster_booking_sum,group_booking_mean,group_booking_sum,predicted_hotel_cluster,actaul_hotel_cluster
8320,9,1,2,50,701,0,1,2138.121578,3,59,...,0.089363,16091,0.057084,442,0.105250,431,0.200000,3,59,59
139438,5,1,2,50,970,0,0,7300.683700,4,42,...,0.089363,16091,0.182796,17,0.122740,577,1.000000,1,42,42
200422,9,1,2,50,1241,0,0,2240.129000,3,5,...,0.089363,16091,0.126394,136,0.069536,572,0.250000,1,16,5
265255,4,1,3,182,46,0,0,7219.419103,2,29,...,0.053390,474,0.063733,265,0.077888,323,0.102941,7,29,62
249345,9,1,2,198,400,0,0,613.090700,3,3,...,0.109852,1948,0.165414,22,0.054622,169,1.000000,1,4,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250980,9,1,2,50,1230,0,0,1855.922500,3,41,...,0.089363,16091,0.086701,517,0.067298,483,0.500000,2,41,41
226091,4,1,2,50,698,0,0,893.673300,3,42,...,0.089363,16091,0.159420,33,0.122740,577,1.000000,1,42,42
17962,9,1,3,129,103,0,1,1877.175685,3,29,...,0.088000,11,0.088000,11,0.077888,323,1.000000,1,29,29
148304,9,1,2,50,637,0,0,1877.175685,3,6,...,0.089363,16091,0.092088,433,0.102819,496,0.384615,5,6,6


In [76]:
clf = make_pipeline(preprocessing.StandardScaler(), 
                    RandomForestClassifier(n_estimators=273,max_depth=10,random_state=0))
np.mean(cross_val_score(clf, X_PCA, y, cv=10))

0.20343472271480065

In [ ]:
##Boosted Tree

In [77]:
from sklearn.ensemble import GradientBoostingClassifier
clf = make_pipeline(preprocessing.StandardScaler(),
                                GradientBoostingClassifier(n_estimators=20, 
                                learning_rate=0.02, max_features=2, max_depth=2, random_state=0))
np.mean(cross_val_score(clf, X_scaled, y, cv=10, scoring='accuracy'))   

0.17178522940833732

In [78]:
clf = make_pipeline(preprocessing.StandardScaler(),
                                GradientBoostingClassifier(n_estimators=20, 
                                learning_rate=0.02, max_features=2, max_depth=2, random_state=0))
np.mean(cross_val_score(clf, X_PCA, y, cv=10, scoring='accuracy'))   

0.0510464053600039

In [79]:
from sklearn.ensemble import GradientBoostingClassifier
clf = make_pipeline(preprocessing.StandardScaler(),
                                GradientBoostingClassifier(n_estimators=20, 
                                learning_rate=0.02, max_features=2, max_depth=2, random_state=0))
np.mean(cross_val_score(clf, X, y, cv=10, scoring='accuracy')) 

0.17178522940833732

In [ ]:
##logistic

In [87]:
clf = make_pipeline(preprocessing.StandardScaler(), LogisticRegression(multi_class='ovr'))
np.mean(cross_val_score(clf, X, y, cv=10))

0.22894248151932856

In [88]:
clf = make_pipeline(preprocessing.StandardScaler(), LogisticRegression(multi_class='ovr'))
np.mean(cross_val_score(clf, X_PCA, y, cv=10))

0.16728642080944453

In [80]:
from sklearn.linear_model import LogisticRegression
clf = make_pipeline(preprocessing.StandardScaler(), LogisticRegression(multi_class='ovr'))
np.mean(cross_val_score(clf, X_scaled, y, cv=10))

0.22894248151932856

In [ ]:
##SVM

In [89]:
from sklearn import svm
clf = make_pipeline(preprocessing.StandardScaler(), svm.SVC(decision_function_shape='ovo'))
np.mean(cross_val_score(clf, X, y, cv=10))

0.26796136631318734

In [86]:
clf = make_pipeline(preprocessing.StandardScaler(), svm.SVC(decision_function_shape='ovo'))
np.mean(cross_val_score(clf, X_PCA, y, cv=10))

0.21630719487342648

In [ ]:
##Neural Network

In [82]:
from sklearn.neural_network import MLPClassifier
clf = make_pipeline(preprocessing.StandardScaler(),MLPClassifier(solver='lbfgs', alpha=1e-5,
                  hidden_layer_sizes=(15, 6),random_state=0))
np.mean(cross_val_score(clf, X, y, cv=10))

0.29904207825096274

In [83]:
clf = make_pipeline(preprocessing.StandardScaler(),MLPClassifier(solver='lbfgs', alpha=1e-5,
                  hidden_layer_sizes=(15, 6),random_state=0))
np.mean(cross_val_score(clf, X_PCA, y, cv=10))

0.2945669800314693

In [84]:
clf = make_pipeline(preprocessing.StandardScaler(),MLPClassifier(solver='lbfgs', alpha=1e-5,
                  hidden_layer_sizes=(15, 6),random_state=0))
np.mean(cross_val_score(clf, X_scaled, y, cv=10))

0.2990005381686408